In [1]:
import pandas as pd
import pyodbc
import os
from unidecode import unidecode
from sqlalchemy import create_engine
import re

In [2]:
# Conexão com banco de dados
sqlgdncli = pyodbc.connect("DRIVER={SQL Server};SERVER=DESKTOP-51H10JN;DATABASE=DW-VendaCombustivel;Trusted_Connection=yes")

In [3]:
def LerArquivoCotacao(caminho, nome_arquivo):

    arquivo = f'{caminho}{nome_arquivo}'
    colunas = [
        'regiao',
        'localidade',
        'municipio',
        'revenda',
        'cnpj',
        'rodovia',
        'numero',
        'complemento',
        'bairro',
        'cep',
        'combustivel',
        'data_coleta',
        'valor_venda',
        'valor_compra',
        'unidade',
        'bandeira'
    ]

    df = pd.read_csv(arquivo, sep=';', names=colunas, header=None, skiprows=1, decimal=',', index_col=False, encoding='latin-1')

    df['caminho'] = caminho
    df['arquivo'] = nome_arquivo

    padrao = re.compile(r"\D(\d{4})-\d{2}\.csv")
    correspondencia = padrao.search(nome_arquivo)
    df['ano'] = correspondencia.group(1)

    print(nome_arquivo)

    return df

In [4]:
# Função para concatenar todos os arquivos presentes no caminho especificado
def ConcatenaArquivosCotacao(caminho):

    # Cria um array contendo o nome de todos os arquivos
    nomes_arquivos = os.listdir(caminho)
    
    # Passa por cada arquivo, chama a função que faz a leitura deles, le e concatena
    for i, y in enumerate(nomes_arquivos):

        df = LerArquivoCotacao(caminho, y)
        if i == 0:
            df_final = df
        else:
            df_final = pd.concat([df_final, df])
    
    return df_final

In [6]:
caminho = 'C:\\Users\\Rhey\\Downloads\\DW-VendasCombustivel\\Arquivos\\Cotacao\\'

df_final = ConcatenaArquivosCotacao(caminho)

ca-2004-02.csv
ca-2005-02.csv
ca-2006-02.csv
ca-2007-02.csv
ca-2008-02.csv
ca-2009-02.csv
ca-2010-02.csv
ca-2011-02.csv
ca-2012-02.csv
ca-2013-02.csv
ca-2014-02.csv
ca-2015-02.csv
ca-2016-02.csv
ca-2017-02.csv
ca-2018-02.csv
ca-2019-02.csv
ca-2020-02.csv
ca-2021-02.csv


In [7]:
colunas_drop = ['municipio', 'revenda', 'cnpj', 'rodovia', 'numero', 'complemento', 'bairro', 'cep', 'data_coleta', 'bandeira']
df_final = df_final.drop(columns=colunas_drop)

In [8]:
# Tentar converter para numérico, tratando erros com 'coerce'
df_final['valor_venda'] = pd.to_numeric(df_final['valor_venda'], errors='coerce')
df_final['valor_compra'] = pd.to_numeric(df_final['valor_compra'], errors='coerce')

# Substituir NaN por um valor padrão (por exemplo, 0)
df_final['valor_venda'] = df_final['valor_venda'].fillna(0)
df_final['valor_compra'] = df_final['valor_compra'].fillna(0)

In [9]:
# Calcular a média por PRODUTO, ESTADO e ANO
media_por_grupo = df_final.groupby(['combustivel', 'localidade', 'ano'])[['valor_venda', 'valor_compra']].mean().reset_index()
media_por_grupo

,combustivel,localidade,ano,valor_venda,valor_compra
0,DIESEL,AC,2004,1.861481,1.156749
1,DIESEL,AC,2005,2.188171,1.577292
2,DIESEL,AC,2006,2.318354,1.612108
3,DIESEL,AC,2007,2.325797,1.436921
4,DIESEL,AC,2008,2.525849,1.616584
...,...,...,...,...,...
2132,GNV,SP,2020,2.892934,0.026206
2133,GNV,SP,2021,3.848536,0.000000
2134,GNV,TO,2008,2.155000,0.927650
2135,GNV,TO,2016,3.049000,0.000000


In [60]:
print(media_por_grupo['localidade'].unique())
print(media_por_grupo['combustivel'].unique())
print(media_por_grupo['valor_venda'].unique())
print(media_por_grupo['valor_compra'].unique())
print(media_por_grupo['ano'].unique())

['AC' 'AL' 'AM' 'AP' 'BA' 'CE' 'DF' 'ES' 'GO' 'MA' 'MG' 'MS' 'MT' 'PA'
 'PB' 'PE' 'PI' 'PR' 'RJ' 'RN' 'RO' 'RR' 'RS' 'SC' 'SE' 'SP' 'TO']
['DIESEL' 'DIESEL S10' 'DIESEL S50' 'ETANOL' 'GASOLINA'
 'GASOLINA ADITIVADA' 'GNV']
[2.18817137 2.31835389 2.32579732 ... 2.155      3.049      3.48      ]
[1.57729231 1.61210786 1.43692057 ... 0.2195762  0.02620633 0.92765   ]
['2005' '2006' '2007' '2008' '2009' '2010' '2011' '2012' '2013' '2014'
 '2015' '2016' '2017' '2018' '2019' '2020' '2021']


In [10]:
#Transfere todos os dataframes para uma tabela em SQL Server
connection_string = f"mssql+pyodbc://DESKTOP-51H10JN/DW-VendaCombustivel?driver=ODBC Driver 17 for SQL Server"
engine = create_engine(connection_string, fast_executemany=True)

media_por_grupo.to_sql(name='FATO_COTACAO', con=engine, index=False, if_exists='replace')

-1

In [62]:
DELETE FROM FATO_COTACAO
WHERE combustivel IN ('DIESEL S10','DIESEL S50','GASOLINA ADITIVADA','GNV')

UPDATE FATO_COTACAO
SET combustivel = 'ETANOL HIDRATADO'
WHERE combustivel = 'ETANOL'

UPDATE FATO_COTACAO
SET combustivel = 'GASOLINA COMUM'
WHERE combustivel = 'GASOLINA'

UPDATE FATO_COTACAO
SET combustivel = 'OLEO DIESEL'
WHERE combustivel = 'DIESEL'

UPDATE A
SET combustivel = b.id
FROM FATO_COTACAO A
INNER JOIN DIM_COMBUSTIVEL B
ON A.combustivel = B.combustivel

UPDATE A
SET localidade = b.id
FROM FATO_COTACAO A
INNER JOIN DIM_LOCALIDADE B
ON A.localidade = B.UF

UPDATE A
SET ano = b.id
FROM FATO_COTACAO A
INNER JOIN DIM_TEMPO B
ON A.ano = B.ano

SyntaxError: invalid syntax (2905641863.py, line 1)